In [14]:
# 수업 시간 함수 인용
import copy

def is_matrix(A):
    m=len(A)
    n=len(A[0])
    for i in range(1,m):
        if len(A[i])!=n:
            return (0,0)
    return (m,n)

def Naive_Gauss(A):
    mA, nA = is_matrix(A)
    if mA == 0:
        print("A는 행렬이 아닙니다.")
        return
    if mA != nA:
        print("A는 정사각 행렬이 아닙니다.")
        return    

    C=copy.deepcopy(A)
    for k in range(mA-1):
        for i in range(k+1,mA):
            xmult = C[i][k]/C[k][k]
            C[i][k] = xmult
            for j in range(k+1,mA):
                C[i][j] -= xmult*C[k][j]
    return C

def matrix_zeros(m,n):
    return [[0 for i in range(n)] for j in range(m)]

def matrix_identity(n):
    return [[1 if i==j else 0 for i in range(n)] for j in range(n)]

def matrix_mult(A,B):
    isA = is_matrix(A)
    isB = is_matrix(B)
    if isA==(0,0):
        print("A는 행렬이 아닙니다.")
        return
    if isB==(0,0):
        print("B는 행렬이 아닙니다.")
        return
    if isA[1]!=isB[0]:
        print("차원이 맞지 않습니다.")
        return
    C=matrix_zeros(isA[0],isB[1])
    for i in range(isA[0]):
        for j in range(isB[1]):
            for k in range(isA[1]):
                C[i][j]+=A[i][k]*B[k][j]
    return C

## 1. Cholesky factorization을 수행하는 함수를 정의

In [15]:
def cholesky_factorization(A):
    mA, nA = is_matrix(A)
    if mA == 0:
        print("A는 행렬이 아닙니다.")
        return
    if mA != nA:
        print("A는 정사각 행렬이 아닙니다.")
        return
    
    C = Naive_Gauss(A)
    L = matrix_identity(mA)
    U = matrix_zeros(mA,nA)
    for i in range(mA):             # Naive_Gauss 함수를 이용하여 L',U' 구하기
        for j in range(i,mA):
            U[i][j]=C[i][j]
        for j in range(i):
            L[i][j]=C[i][j]

    D = matrix_identity(mA)         # U행렬을 D,U으로 분해 그러면 U=L'^T (symmetric positive definite)
    for i in range(mA):
        tmp=U[i][i]
        D[i][i]=tmp
        for j in range(i,mA):
            U[i][j]/=tmp
    
    for i in range(mA):             # D행렬은 diagonal matrix이기 때문에 diagonla entry만 루트 씌우기 (D=D'D')
        D[i][i]=D[i][i]**0.5

    L = matrix_mult(L,D)            # L'행렬과 D'행렬을 곱하여 새로운 L행렬 구하기 그러면 A = LL^T (symmetric positive definite)
    U = matrix_mult(D,U)
    return L,U

In [16]:
A = [[4,12,-16],[12,37,-43],[-16,-43,98]]
cholesky_factorization(A)

([[2.0, 0.0, 0.0], [6.0, 1.0, 0.0], [-8.0, 5.0, 3.0]],
 [[2.0, 6.0, -8.0], [0.0, 1.0, 5.0], [0.0, 0.0, 3.0]])

In [17]:
A=[[4,-1,0],[-1,4,-1],[0,-1,4]]
cholesky_factorization(A)

([[2.0, 0.0, 0.0],
  [-0.5, 1.9364916731037085, 0.0],
  [0.0, -0.5163977794943223, 1.9321835661585918]],
 [[2.0, -0.5, 0.0],
  [0.0, 1.9364916731037085, -0.5163977794943223],
  [0.0, 0.0, 1.9321835661585918]])

## 2. Cholesky factorization을 통해 구한 L 행렬을 이용하여 시스템을 해결하는 Solve 함수 정의

In [18]:
def solve_cholesky(L,b):
    mA, nA = is_matrix(A)
    if mA == 0:
        print("A는 행렬이 아닙니다.")
        return
    if mA != nA:
        print("A는 정사각 행렬이 아닙니다.")
        return
    
    L,U=cholesky_factorization(A)     # cholesky_factorization을 이용하여 L행렬
    y=[0 for i in range(mA)]
    x=[0 for i in range(mA)]

    for i in range(mA):             # L^Tx=y 로 하고 Ly=b 시스템을 풀어 y 구하기
        y[i]=b[i]/L[i][i]
        for j in range(i):
            y[i]-=L[i][j]*y[j]/L[i][i]
    
    for i in range(mA-1,-1,-1):     # 구한 y를 이용하여 L^Tx=y 시스템을 풀어 x 구하기
        x[i]=y[i]/L[i][i]
        for j in range(i+1,mA):
            x[i]-=L[j][i]*x[j]/L[i][i]
            
    return x

In [19]:
A = [[4,12,-16],[12,37,-43],[-16,-43,98]]
b = [0,6,39]
solve_cholesky(A,b)

[1.0, 1.0, 1.0]

In [20]:
A=[[4,-1,0],[-1,4,-1],[0,-1,4]]
b=[7,-1,11]
solve_cholesky(A,b)

[2.0, 1.0000000000000002, 3.0000000000000004]